In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[8]") \
                    .appName('sparkedu') \
                    .getOrCreate()

In [3]:
from ast import literal_eval
rdd = spark.read.text("output/(수정)최종레시피1.csv", converters={'ingredient' : literal_eval, 'seasoning' : literal_eval, 'howto' : literal_eval})
print(type(rdd))
print(rdd.collect()[1])

TypeError: text() got an unexpected keyword argument 'converters'

In [3]:
# AWS-LINUX

import findspark
findspark.init("/opt/spark")

In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.getOrCreate()
spark

In [6]:
dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
rdd=spark.sparkContext.parallelize(dataList)
print(rdd)
print(type(rdd))
print(rdd.collect())

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262
<class 'pyspark.rdd.RDD'>
[('Java', 20000), ('Python', 100000), ('Scala', 3000)]


In [7]:
print(spark.version)

3.0.1


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [4]:
recipe_df = spark.read.csv("output/(수정)최종레시피1.csv",header = 'True',inferSchema='True')

In [5]:
passengers_count = recipe_df.count()

In [6]:
print(passengers_count)

9895


In [17]:
recipe_df.show(5)

+----------------------------------+--------------------------+---------------------------------+--------------------------+--------------------------------+-----------------------------+-----+--------------------+----------+
|                              name|                ingredient|                  ingredient_name|                 seasoning|                  seasoning_name|                        howto|views|                 img|recipe_num|
+----------------------------------+--------------------------+---------------------------------+--------------------------+--------------------------------+-----------------------------+-----+--------------------+----------+
|손쉽게 만드는 소불고기덮밥 레시...|  ['소불고기', '200g', ...|               소불고기 양파 대파| ['간장', '4스푼', '물o...|   간장 다진마늘 올리고당 참기름|['1. 양파와 대파는 미리 썰...|  587|https://recipe1.e...|   6947317|
|   사과를 활용한 요리 / 프랑스 ...| ['계란', '1개', '설탕'...|계란 설탕 우유 생크림 찹쌀가루...|                        []|                            null|  ['1. 볼에 계란, 설탕, 우..

In [18]:
print(type(recipe_df['howto'][2]))

<class 'pyspark.sql.column.Column'>


In [19]:
recipe_df.printSchema()
recipe_df.show()

root
 |-- name: string (nullable = true)
 |-- ingredient: string (nullable = true)
 |-- ingredient_name: string (nullable = true)
 |-- seasoning: string (nullable = true)
 |-- seasoning_name: string (nullable = true)
 |-- howto: string (nullable = true)
 |-- views: string (nullable = true)
 |-- img: string (nullable = true)
 |-- recipe_num: string (nullable = true)

+-----------------------------------+-----------------------------+---------------------------------+---------------------------+-----------------------------------+------------------------------+-----+--------------------+----------+
|                               name|                   ingredient|                  ingredient_name|                  seasoning|                     seasoning_name|                         howto|views|                 img|recipe_num|
+-----------------------------------+-----------------------------+---------------------------------+---------------------------+--------------------------------

In [8]:
data = recipe_df.select('ingredient').collect()
print(type(data[0]))
print(data[0])

<class 'pyspark.sql.types.Row'>
Row(ingredient="['소불고기', '200g', '양파', '1/2개', '대파', '3개']")


In [48]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# data = pd.read_csv('../smart_refrigerator/output/test.csv', low_memory=False)
data2 = pd.read_csv('../smart_refrigerator/output/냉장고재료test용.csv', low_memory=False)
data = recipe_df.select('ingredient_name').collect()
print(data[0])
# recipe_df.select(col('ingredient_name')) = recipe_df.select(col('ingredient_name')).fillna('')

tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(data)

print(tfidf_matrix.shape)

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(recipe_df.index, index=recipe_df['name'])
print(indices.head())

def get_recommendations(food, cosine_sim=cosine_sim):
    idx = indices[food]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    food_indices = [i[0] for i in sim_scores]

    return recipe_df['name'].iloc[food_indices]

get_recommendations('소고기 덮밥~*')

Row(ingredient_name='소불고기 양파 대파')


AttributeError: lower